##구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
FACE_width=46
FACE_height=56
FACE_size=(FACE_width,FACE_height)
FACE_channels=3

In [3]:
IRIS_width=192
IRIS_height=144
IRIS_size=(IRIS_width,IRIS_height)
IRIS_channels=3

In [4]:
IMAGE_width=100
IMAGE_height=100
IMAGE_size=(IMAGE_width,IMAGE_height)
IMAGE_channels=3

## 파일 경로 지정

In [5]:
train_path='/gdrive/My Drive/생체인증보안/멀티모달/train_set'
test_path='/gdrive/My Drive/생체인증보안/멀티모달/test_set'

## 라이브러리 import

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras.preprocessing import image
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import tensorflow as tf
import random
import glob, os
import cv2
from PIL import Image

## Augmentation 후 저장

In [ ]:
# 랜덤시드 고정시키기
np.random.seed(50)

from keras.preprocessing.image import ImageDataGenerator, img_to_array

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(brightness_range=[0.8,1.2],
                                  zoom_range=[0.9, 1.1],
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  fill_mode='constant')
 

In [ ]:
%cd '/gdrive/My Drive/생체인증보안/멀티모달/train_set'
nowpath='/gdrive/My Drive/생체인증보안/멀티모달/train_set'

for filename in os.listdir(nowpath):
  dtype=filename[4:8]
  if dtype=="iris":
    img=Image.open(filename)
    img = img_to_array(img)
    img = img.reshape((1,) + img.shape)
    i = 0
  # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
    for batch in data_aug_gen.flow(img, batch_size=1, save_to_dir='./', save_prefix=filename, save_format='png'):
      i += 1
      if i >4: 
           break

/gdrive/My Drive/생체인증보안/멀티모달/train_set


In [ ]:
images=sorted(glob.glob("*"))

print(len(images))

3072


##데이터 불러오기

In [ ]:
import natsort
f_x_data=[]
f_y_data=[]

i_x_data=[]
i_y_data=[]

fname=[]
for filename in os.listdir(train_path):
  fname.append(filename)

fname=natsort.natsorted(fname)

for file_name in fname:
    file_path='{}/{}'.format(train_path, file_name)
    image=Image.open(file_path).convert("RGB")

    name=file_name[0:3]
    y_name=int(name)
    dtype=file_name[4:8]

    if dtype=="face":
      image=image.resize((FACE_width,FACE_height))
      np_image=np.asarray(image)
      np_image=np_image/255
      f_x_data.append(np_image)
      f_y_data.append(y_name)
    elif dtype=="iris":
      image=image.resize((IRIS_width,IRIS_height))
      np_image=np.asarray(image)
      np_image=np_image/255
      i_x_data.append(np_image)
      i_y_data.append(y_name)
  


In [ ]:
f_x_data = np.array(f_x_data)
f_y_data = np.array(f_y_data)

i_x_data = np.array(i_x_data)
i_y_data = np.array(i_y_data)

In [ ]:
f_x_data.shape

(1536, 56, 46, 3)

In [ ]:
i_x_data.shape

(1536, 144, 192, 3)

In [ ]:
print(f_y_data)

[ 0  0  0 ... 63 63 63]


In [ ]:
print(i_y_data)

[ 0  0  0 ... 63 63 63]


In [ ]:
#np.save("/gdrive/My Drive/생체인증보안/멀티모달/x_data.npy",x_data)
#np.save("/gdrive/My Drive/생체인증보안/멀티모달/y_data.npy",y_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/f_x_data.npy",f_x_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/f_y_data.npy",f_y_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/i_x_data.npy",i_x_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/i_y_data.npy",i_y_data)

In [7]:
#x_data = np.load('/gdrive/My Drive/생체인증보안/홍채/x_data.npy')
#y_data = np.load('/gdrive/My Drive/생체인증보안/홍채/y_data.npy')
f_x_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/f_x_data.npy')
f_y_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/f_y_data.npy')
i_x_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/i_x_data.npy')
i_y_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/i_y_data.npy')

##모델링

In [8]:
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201, DenseNet121
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import losses, optimizers, activations, metrics

base_model=DenseNet121(input_shape=(FACE_height,FACE_width,FACE_channels),include_top=False)

x = base_model.output
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)

predictions = Dense(64, activation='softmax')(out)
facemodel = Model(inputs=base_model.input, outputs=predictions)

adam=optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
facemodel.compile(optimizer=adam, loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

facemodel.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 46, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 62, 52, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 28, 23, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 28, 23, 64)   256         conv1/conv[0][0]                 
_____________________________

In [9]:
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201, DenseNet121
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import losses, optimizers, activations, metrics

base_model=DenseNet121(input_shape=(IRIS_height,IRIS_width,IRIS_channels),include_top=False)

x = base_model.output
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)

predictions = Dense(64, activation='softmax')(out)
irismodel = Model(inputs=base_model.input, outputs=predictions)

adam=optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
irismodel.compile(optimizer=adam, loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

irismodel.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 144, 192, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 150, 198, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 72, 96, 64)   9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 72, 96, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________________________________

##Callback

In [10]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#조기종료
earlystop=EarlyStopping(monitor='sparse_categorical_accuracy',patience=10)

#학습율 조정
learning_rate_reduction=ReduceLROnPlateau(monitor='sparse_categorical_accuracy',
                                          patience=2,
                                          verbose=1,
                                          factor=0.5,
                                          min_lr=0.000001)
#callback 설정
callback=[earlystop,learning_rate_reduction]

##모델 학습(Cross-Validation)

In [11]:
#얼굴
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=5)
cvscores = []

for train_idx,val_idx in kfold.split(f_x_data,f_y_data):
  x_train,x_val=f_x_data[train_idx],f_x_data[val_idx]
  y_train,y_val=f_y_data[train_idx],f_y_data[val_idx]

  facemodel.fit(x_train,y_train,epochs=6,batch_size=10,verbose=1,callbacks=callback)
  
  scores = facemodel.evaluate(x_val, y_val, verbose=1)
  print("Acc: ",(scores[1]*100))
  cvscores.append(scores[1] * 100)

Epoch 1/6
123/123 [==============================] - 49s 62ms/step - loss: 5.0248 - sparse_categorical_accuracy: 0.0497
Epoch 2/6
123/123 [==============================] - 6s 49ms/step - loss: 2.9360 - sparse_categorical_accuracy: 0.2777
Epoch 3/6
123/123 [==============================] - 6s 49ms/step - loss: 1.8286 - sparse_categorical_accuracy: 0.5309
Epoch 4/6
123/123 [==============================] - 6s 50ms/step - loss: 1.0230 - sparse_categorical_accuracy: 0.7321
Epoch 5/6
123/123 [==============================] - 6s 50ms/step - loss: 0.6185 - sparse_categorical_accuracy: 0.8485
Epoch 6/6
10/10 [==============================] - 3s 98ms/step - loss: 0.0512 - sparse_categorical_accuracy: 0.9935
Acc:  99.35064911842346
Epoch 1/6
123/123 [==============================] - 8s 63ms/step - loss: 0.3615 - sparse_categorical_accuracy: 0.9138
Epoch 2/6
123/123 [==============================] - 6s 50ms/step - loss: 0.2632 - sparse_categorical_accuracy: 0.9414
Epoch 3/6
123/123 [======

In [12]:
print("평균 검증 Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

평균 검증 Accuracy: 99.80% (+/- 0.26%)


In [13]:
#홍채
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=3)
cvscores = []

for train_idx,val_idx in kfold.split(i_x_data,i_y_data):
  x_train,x_val=i_x_data[train_idx],i_x_data[val_idx]
  y_train,y_val=i_y_data[train_idx],i_y_data[val_idx]

  irismodel.fit(x_train,y_train,epochs=6,batch_size=16,verbose=1,callbacks=callback)
  
  scores = irismodel.evaluate(x_val, y_val, verbose=1)
  print("Acc: ",(scores[1]*100))
  cvscores.append(scores[1] * 100)

Epoch 1/6
64/64 [==============================] - 18s 110ms/step - loss: 2.5698 - sparse_categorical_accuracy: 0.5176
Epoch 2/6
64/64 [==============================] - 7s 110ms/step - loss: 0.0900 - sparse_categorical_accuracy: 0.9766
Epoch 3/6
64/64 [==============================] - 7s 110ms/step - loss: 0.0213 - sparse_categorical_accuracy: 0.9961
Epoch 4/6
64/64 [==============================] - 7s 110ms/step - loss: 0.0065 - sparse_categorical_accuracy: 1.0000
Epoch 5/6
64/64 [==============================] - 7s 111ms/step - loss: 0.0049 - sparse_categorical_accuracy: 1.0000
Epoch 6/6
64/64 [==============================] - 7s 111ms/step - loss: 0.0033 - sparse_categorical_accuracy: 1.0000

Epoch 00006: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
16/16 [==============================] - 4s 60ms/step - loss: 0.0432 - sparse_categorical_accuracy: 0.9922
Acc:  99.21875
Epoch 1/6
64/64 [==============================] - 7s 110ms/step - loss: 0.0570 - sparse

In [14]:
print("평균 검증 Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

평균 검증 Accuracy: 99.74% (+/- 0.37%)


In [ ]:
%cd '/gdrive/My Drive/생체인증보안/멀티모달/'
facemodel.save('face.h5')
irismodel.save('iris.h5')

##모델 성능 지표</br>
학습 데이터 중 split한 validation 데이터 사용

###얼굴

In [15]:
f_y_train_pred=facemodel.predict(f_x_data)
f_y_train_pred

array([[1.00000000e+00, 1.52382672e-11, 1.64314368e-10, ...,
        4.14547840e-11, 1.86131405e-11, 8.10977847e-12],
       [1.00000000e+00, 3.11715714e-14, 7.82971913e-14, ...,
        1.52520927e-13, 3.22114839e-13, 1.76954263e-14],
       [1.00000000e+00, 6.64184853e-12, 2.71759264e-11, ...,
        3.96346497e-12, 1.59483191e-11, 2.45296253e-12],
       ...,
       [6.05584871e-09, 1.03717479e-08, 4.79033559e-08, ...,
        1.99170927e-06, 1.19375542e-07, 9.99982715e-01],
       [1.64048120e-09, 1.83016340e-08, 4.49302391e-08, ...,
        1.10044994e-06, 9.16042140e-08, 9.99991417e-01],
       [3.79550447e-10, 2.16151341e-09, 1.46608690e-08, ...,
        1.05676314e-07, 1.20293322e-08, 9.99998331e-01]], dtype=float32)

In [16]:
print(f_y_data)

[ 0  0  0 ... 63 63 63]


In [17]:
f_y_pred_train=[]
for i in range(len(f_y_data)):
  n=np.argmax(f_y_train_pred[i])
  f_y_pred_train.append(n)

In [18]:
print(f_y_pred_train)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,

In [19]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(f_y_data,f_y_pred_train)                         
precision = precision_score(f_y_data,f_y_pred_train,average='micro') 
recall = recall_score(f_y_data,f_y_pred_train, average='micro')      
f1 = f1_score(f_y_data,f_y_pred_train, average='micro')

print('정확도: {0:.6f}, 정밀도: {1:.6f}, 재현율: {2:.6f}, F1-score: {3:.6f}'.
      format(accuracy, precision, recall, f1))

정확도: 1.000000, 정밀도: 1.000000, 재현율: 1.000000, F1-score: 1.000000


###홍채

In [20]:
i_y_train_pred=irismodel.predict(i_x_data)
i_y_train_pred

array([[9.9999952e-01, 5.0094706e-10, 9.8514141e-10, ..., 1.6802532e-09,
        5.3890969e-10, 8.3451600e-13],
       [1.0000000e+00, 4.0426551e-12, 4.6663659e-11, ..., 7.5592352e-11,
        2.2629119e-11, 1.3635793e-13],
       [9.9999797e-01, 2.3887414e-09, 5.0051483e-09, ..., 2.5376325e-09,
        3.6090497e-09, 9.0366028e-13],
       ...,
       [3.4837637e-08, 4.5947171e-09, 1.2165602e-08, ..., 1.2866541e-10,
        2.7954359e-08, 9.9999797e-01],
       [5.3201632e-08, 7.3483983e-09, 2.2835215e-08, ..., 9.8052251e-09,
        1.1891221e-07, 9.9998391e-01],
       [5.3201632e-08, 7.3483983e-09, 2.2835215e-08, ..., 9.8052251e-09,
        1.1891221e-07, 9.9998391e-01]], dtype=float32)

In [21]:
print(i_y_data)

[ 0  0  0 ... 63 63 63]


In [22]:
i_y_pred_train=[]
for i in range(len(i_y_data)):
  n=np.argmax(i_y_train_pred[i])
  i_y_pred_train.append(n)

In [23]:
print(i_y_pred_train)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,

In [24]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(i_y_data,i_y_pred_train)                         
precision = precision_score(i_y_data,i_y_pred_train,average='micro') 
recall = recall_score(i_y_data,i_y_pred_train, average='micro')      
f1 = f1_score(i_y_data,i_y_pred_train, average='micro')

print('정확도: {0:.6f}, 정밀도: {1:.6f}, 재현율: {2:.6f}, F1-score: {3:.6f}'.
      format(accuracy, precision, recall, f1))

정확도: 1.000000, 정밀도: 1.000000, 재현율: 1.000000, F1-score: 1.000000


##테스트

In [25]:
import natsort
%cd '/gdrive/My Drive/생체인증보안/멀티모달/test_set'

fname=[]
f_name=[]
i_name=[]

f_x_test=[]
f_y_test=[]
i_x_test=[]
i_y_test=[]

for filename in os.listdir(test_path):
  fname.append(filename)

fname=natsort.natsorted(fname)

for filename in fname:
  img=Image.open(filename).convert("RGB")
  print(filename)
  if 'face' in filename:
    img=img.resize((FACE_width,FACE_height))
    np_image=np.asarray(img)
    np_image=np_image/255
    f_x_test.append(np_image)
    f_name.append(filename)
  elif 'iris' in filename:
    img=img.resize((IRIS_width,IRIS_height))
    np_image=np.asarray(img)
    np_image=np_image/255
    i_x_test.append(np_image)
    i_name.append(filename)
  
f_x_test = np.array(f_x_test)
i_x_test = np.array(i_x_test)

/gdrive/My Drive/생체인증보안/멀티모달/test_set
0_face.png
0_iris.png
1_face.png
1_iris.png
2_face.png
2_iris.png
3_face.png
3_iris.png
4_face.png
4_iris.png
5_face.png
5_iris.png
6_face.png
6_iris.png
7_face.png
7_iris.png
8_face.png
8_iris.png
9_face.png
9_iris.png
10_face.png
10_iris.png
11_face.png
11_iris.png
12_face.png
12_iris.png
13_face.png
13_iris.png
14_face.png
14_iris.png
15_face.png
15_iris.png
16_face.png
16_iris.png
17_face.png
17_iris.png
18_face.png
18_iris.png
19_face.png
19_iris.png
20_face.png
20_iris.png
21_face.png
21_iris.png
22_face.png
22_iris.png
23_face.png
23_iris.png
24_face.png
24_iris.png
25_face.png
25_iris.png
26_face.png
26_iris.png
27_face.png
27_iris.png
28_face.png
28_iris.png
29_face.png
29_iris.png
30_face.png
30_iris.png
31_face.png
31_iris.png
32_face.png
32_iris.png
33_face.png
33_iris.png
34_face.png
34_iris.png
35_face.png
35_iris.png
36_face.png
36_iris.png
37_face.png
37_iris.png
38_face.png
38_iris.png
39_face.png
39_iris.png
40_face.png
40_iris.pn

In [26]:
f_x_test.shape

(64, 56, 46, 3)

In [27]:
i_x_test.shape

(64, 144, 192, 3)

In [ ]:
np.save("/gdrive/My Drive/생체인증보안/멀티모달/f_x_test.npy",f_x_test)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/i_x_test.npy",i_x_test)

In [ ]:
f_x_test = np.load('/gdrive/My Drive/생체인증보안/멀티모달/f_x_test.npy')
i_x_test = np.load('/gdrive/My Drive/생체인증보안/멀티모달/f_x_test.npy')

###얼굴 예측

In [28]:
f_y_pred=facemodel.predict(f_x_test)
print(f_y_pred)

[[8.42805730e-06 4.37106337e-06 1.99746173e-05 ... 9.94650782e-06
  1.75825642e-06 1.17333198e-04]
 [1.65769109e-07 4.81215693e-07 2.28249482e-05 ... 1.00698159e-07
  5.16546557e-08 3.29865600e-07]
 [2.35442638e-10 2.09166004e-13 1.91319724e-10 ... 1.07202745e-11
  2.94949135e-11 3.63677491e-12]
 ...
 [8.72008101e-08 9.41090946e-07 1.95011296e-07 ... 4.84141165e-05
  3.80143229e-06 8.10597885e-06]
 [1.79837585e-07 1.45699701e-07 1.02363558e-07 ... 7.74871989e-09
  7.68497830e-06 6.32059127e-10]
 [1.65461051e-06 4.48897117e-05 1.09501218e-03 ... 2.48410288e-07
  2.20193524e-05 9.73876922e-07]]


In [29]:
fresult=[]
fdata=[]
for i in range(64):
  n=np.argmax(f_y_pred[i])
  fdata.append(f_y_pred[i][n])
  fresult.append(n)

In [30]:
print(fresult)

[27, 31, 22, 57, 19, 45, 5, 1, 8, 7, 36, 11, 46, 50, 1, 61, 49, 59, 56, 13, 37, 30, 25, 0, 41, 58, 9, 20, 36, 47, 42, 24, 3, 6, 57, 63, 21, 16, 34, 11, 35, 32, 40, 62, 19, 52, 33, 15, 13, 29, 17, 26, 53, 28, 9, 14, 43, 60, 4, 39, 38, 23, 51, 44]


In [31]:
flist_df = pd.DataFrame({'Image':f_name,'Answer':fresult})
flist_df.head(20)

,Image,Answer
0,0_face.png,27
1,1_face.png,31
2,2_face.png,22
3,3_face.png,57
4,4_face.png,19
5,5_face.png,45
6,6_face.png,5
7,7_face.png,1
8,8_face.png,8
9,9_face.png,7


###홍채 예측

In [32]:
i_y_pred=irismodel.predict(i_x_test)
print(i_y_pred)

[[1.4168058e-06 9.6382814e-07 7.7924290e-07 ... 6.5930816e-04
  9.7440150e-08 2.7263877e-06]
 [9.9285797e-05 1.1592983e-04 7.6923091e-03 ... 6.4788896e-05
  4.1170287e-04 1.9361539e-03]
 [2.1344999e-08 2.0139517e-07 2.8835693e-08 ... 2.3725987e-07
  2.2853837e-09 1.0268590e-08]
 ...
 [2.4868757e-07 2.5155617e-07 3.4958305e-07 ... 3.1014646e-05
  1.0715318e-07 1.4329002e-07]
 [3.1198385e-06 3.0922219e-08 1.4540912e-08 ... 9.5619946e-08
  5.8222863e-08 1.2819021e-07]
 [2.6798405e-06 1.5815915e-06 6.4717700e-05 ... 3.3065692e-09
  2.0888390e-07 2.1418499e-07]]


In [33]:
iresult=[]
idata=[]
for i in range(64):
  n=np.argmax(i_y_pred[i])
  idata.append(i_y_pred[i][n])
  iresult.append(n)

In [34]:
print(iresult)

[27, 31, 22, 57, 54, 45, 5, 12, 8, 7, 36, 2, 46, 50, 10, 61, 49, 59, 56, 13, 37, 30, 25, 0, 41, 58, 9, 20, 48, 47, 42, 24, 3, 6, 55, 63, 21, 16, 34, 11, 35, 32, 40, 62, 19, 52, 33, 15, 1, 29, 17, 26, 53, 28, 18, 14, 43, 60, 4, 39, 38, 23, 51, 44]


In [35]:
ilist_df = pd.DataFrame({'Image':i_name,'Answer':iresult})
ilist_df.head(20)

,Image,Answer
0,0_iris.png,27
1,1_iris.png,31
2,2_iris.png,22
3,3_iris.png,57
4,4_iris.png,54
5,5_iris.png,45
6,6_iris.png,5
7,7_iris.png,12
8,8_iris.png,8
9,9_iris.png,7


###최종 예측

In [36]:
result=[]
for i in range(64):
  if fdata[i]>idata[i]:
    result.append(fresult[i])
  else:
    result.append(iresult[i])

print(result)

[27, 31, 22, 57, 54, 45, 5, 12, 8, 7, 36, 2, 46, 50, 10, 61, 49, 59, 56, 13, 37, 30, 25, 0, 41, 58, 9, 20, 48, 47, 42, 24, 3, 6, 55, 63, 21, 16, 34, 11, 35, 32, 40, 62, 19, 52, 33, 15, 1, 29, 17, 26, 53, 28, 18, 14, 43, 60, 4, 39, 38, 23, 51, 44]


In [37]:
list_df = pd.DataFrame({'fImage':f_name,'iImage':i_name,'fAnswer':fresult,'iAnswer':iresult,'Answer':result})
list_df.head(20)

,fImage,iImage,fAnswer,iAnswer,Answer
0,0_face.png,0_iris.png,27,27,27
1,1_face.png,1_iris.png,31,31,31
2,2_face.png,2_iris.png,22,22,22
3,3_face.png,3_iris.png,57,57,57
4,4_face.png,4_iris.png,19,54,54
5,5_face.png,5_iris.png,45,45,45
6,6_face.png,6_iris.png,5,5,5
7,7_face.png,7_iris.png,1,12,12
8,8_face.png,8_iris.png,8,8,8
9,9_face.png,9_iris.png,7,7,7


In [38]:
list_df.to_csv("/gdrive/My Drive/생체인증보안/멀티모달/result2-모델2개.csv")